# Пайплайн № 7
### База знаний: FAQ, Реальные Кейсы, Пользовательское соглашение и Условия размещения

### Защита от галлюцинаций: если не уверены в ответе - возвращаем "Нет ответа", но отдаём два классификатора.

### Добавили Ranker, который более тщательно ранжирует релевантные вопросы

### Добавили LLM

### Установка необходимых библиотек

In [1]:
# !export LLAMA_CUBLAS=1
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python --verbose
# !pip install llama-cpp-haystack

In [2]:
%%bash

pip install haystack-ai
pip install "datasets>=2.6.1"
pip install "sentence-transformers>=3.0.0"
pip install pymorphy2
pip install catboost
pip install python-docx

### Подготовка данных

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cp -r drive/MyDrive/HacksAI/Russia24/Dataset Dataset/

Эталонная База Знаний FAQ

In [5]:
import pandas as pd
import json

know_base_path = 'Dataset/01_База_знаний.xlsx'
df_base = pd.read_excel(know_base_path)

df_base.head()

,Тема,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Что нельзя публиковать на RUTUBE,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
2,Почему могут отключить монетизацию на видео и ...,Почему могут отключить монетизацию из-за искус...,Монетизация на RUTUBE зависит в том числе от к...,МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Почему могут отключить монетизацию на видео и ...,"Для каких статусов доступна монетизация, и поч...","Монетизацию на RUTUBE можно подключить, если в...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,Авторское право,Какой контент можно использовать для монетизац...,"То, что вы создали сами: видео, которое вы сня...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


База Реальных Кейсов

In [6]:
real_case_path = 'Dataset/02_Реальные_кейсы.xlsx'
df_case = pd.read_excel(real_case_path)

df_case.head()

,Вопрос пользователя,Ответ сотрудника,Вопрос из БЗ,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
3,Что запрещено в монетизации и что можно выклад...,"Здравствуйте!\nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации
4,"Чтобы не отключали монетизацию, надо, чтобы я ...","Для монетизации можно использовать то, что вы ...",Почему могут отключить монетизацию из-за автор...,"Монетизация может отключиться, если на вашем к...",МОНЕТИЗАЦИЯ,Отключение/подключение монетизации


Объединим обе базы для полее полного и объективного поиска

In [7]:
combined_column = pd.concat([df_base['Вопрос из БЗ'], df_case['Вопрос пользователя']], ignore_index=True)

df_base_rest = df_base.drop(columns=['Вопрос из БЗ'])
df_case_rest = df_case.drop(columns=['Вопрос пользователя'])

combined_rest = pd.concat([df_base_rest, df_case_rest], ignore_index=True)

final_df = combined_rest.copy()
final_df['Вопрос'] = combined_column

final_df = final_df[['Вопрос'] + [col for col in final_df.columns if col != 'Вопрос']]
final_df = final_df.drop(columns=["Тема"])

final_df.tail()

,Вопрос,Ответ из БЗ,Классификатор 1 уровня,Классификатор 2 уровня,Ответ сотрудника,Вопрос из БЗ
1122,Добрый день! Как добавить таймкоды в видео?,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...,ВИДЕО,Загрузка видео,Характеристики эпизодов:\n\nОпределяются по та...,Как добавить тайм-коды под видео?
1123,Как добавить временные интервалы,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...,ВИДЕО,Загрузка видео,Перейдите в раздел «Видео» в Студии RUTUBE.\nН...,Как добавить тайм-коды под видео?
1124,В каком формате добавляются таймкоды?,Главное — соблюдать несколько правил.\n\nТайм-...,ВИДЕО,Загрузка видео,Формат: минуты:секунды или часы:минуты:секунды...,В каком формате нужно добавлять тайм-коды под ...
1125,Как мне правильно указать тайм-код?,Главное — соблюдать несколько правил.\n\nТайм-...,ВИДЕО,Загрузка видео,Можете указать минуты:секунды или часы:минуты:...,В каком формате нужно добавлять тайм-коды под ...
1126,Как правильно указывать время под видео?,Главное — соблюдать несколько правил.\n\nТайм-...,ВИДЕО,Загрузка видео,Тайм-код должен отображать время в формате мин...,В каком формате нужно добавлять тайм-коды под ...


In [8]:
import docx
from haystack import Document

docs = [Document(content=row["Вопрос"], meta={"source": "FAQ", "idx": index}) for index, row in final_df.iterrows()]

Добавим документы: Пользовательское соглашение и Условия размещения

In [9]:
my_doc = docx.Document('Dataset/03_ГЕНЕРАЛЬНОЕ_ПОЛЬЗОВАТЕЛЬСКОЕ_СОГЛАШЕНИЕ_RUTUBE.docx')
para_list = [paragraph.text for paragraph in my_doc.paragraphs if paragraph.text != ""]

para_tuples_list = []

for paragraph in para_list:
    parts = paragraph.split("    ")
    if len(parts) > 1:
        point, content = parts[0].strip(), parts[-1].strip()
        # Разбиваем содержание на чанки по 500 символов
        for i in range(0, len(content), 2048):
            chunk = content[i:i+2048]
            para_tuples_list.append((point, chunk))


for index, paragraph in para_tuples_list:
  docs.append(Document(content=paragraph, meta={"source": "Agreement", "idx": index}))

In [10]:
para_tuples_list

[('1.', 'ПРЕАМБУЛА.'),
 ('1.1.',
  'Настоящее Пользовательское соглашение, именуемое в дальнейшем «Соглашение», представляет собой публичную оферту в адрес любого совершеннолетнего физического лица, прошедшего Процедуру регистрации/ авторизацию в RUTUBE в порядке предусмотренном, Соглашением, и использующим какие-либо из предоставленных возможностей RUTUBE от Общества с ограниченной ответственностью «РУФОРМ» (ИНН 7714886605, ОГРН 1127747031840), зарегистрированного по адресу: 101000, г. Москва, ул. Мясницкая, д.46, стр. 7, офис I, комн. 16, именуемого в дальнейшем «Администрация», являющегося Стороной первого российского Меморандума о сотрудничестве в сфере охраны исключительных прав в эпоху развития цифровых технологий от «01» ноября 2018 года (далее по тексту – Меморандум), а также лицом, на законных основаниях управляющим RUTUBE и предоставляющим неограниченному кругу лиц (Пользователям/ Авторам и/или по отдельным письменным соглашениям - Правообладателям) возможность посредством се

In [11]:
my_doc = docx.Document('Dataset/04_УСЛОВИЯ РАЗМЕЩЕНИЯ КОНТЕНТА.docx')
para_list = [paragraph.text for paragraph in my_doc.paragraphs if paragraph.text != ""]

para_tuples_list = []

for paragraph in para_list:
    parts = paragraph.split(".")
    idx = parts[0].strip() if len(parts[0].strip()) <= 2 else ""
    content = ".".join(parts[1:]).strip() if idx else paragraph.strip()

    # Разбиваем содержание на чанки
    for i in range(0, len(content), 2048):
        chunk = content[i:i+2048]
        para_tuples_list.append((idx, chunk))

for index, paragraph in para_tuples_list:
  docs.append(Document(content=paragraph, meta={"source": "Content", "idx": index}))

In [12]:
len(docs)

1322

### Инициализируем Трансформер для построения эмбеддинга вопросов

In [13]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.utils import ComponentDevice, Device

device = ComponentDevice.from_str("cuda:0")

embed_model = "cointegrated/LaBSE-en-ru"
# embed_model = "intfloat/e5-large-v2"

doc_embedder = SentenceTransformersDocumentEmbedder(
    model=embed_model,
    device=device
    )

doc_embedder.warm_up()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Запишем вопросы в векторное хранилище

In [14]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

docs_with_embeddings = doc_embedder.run(docs)
unique_docs = {doc.id: doc for doc in docs_with_embeddings["documents"]}.values()
document_store.write_documents(list(unique_docs))

Batches:   0%|          | 0/42 [00:00<?, ?it/s]

1322

### Инициализируем Трансформер для поиска по эмбеддингам вопросов
он должен совпадать с трансформером, на основе которого строились эмбеддинги

In [15]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(
    model=embed_model,
    device=device,
)

### Инициализируем Retriever

In [16]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store, top_k=100)

### Добавим LLM модель, которой на вход мы будем подавать весь найденный в RAG контекст

In [17]:
from transformers import pipeline
GROUNDED_SYSTEM_PROMPT = "Your task is to answer the user's questions using only the information from the provided documents."

pipe = pipeline("text-generation", model="Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24", device="cuda:0")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Собираем пайплайн

In [18]:
from haystack import Pipeline

from haystack.components.rankers import TransformersSimilarityRanker

ranker = TransformersSimilarityRanker(
    model="DiTy/cross-encoder-russian-msmarco",
    top_k=3,
    device=device
    )

basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("ranker", ranker)

# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "ranker")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - ranker: TransformersSimilarityRanker
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> ranker.documents (List[Document])

Добавим словарь, по которому будем заменять англицизмы на действительно английские слова (т. к. в базе они хранятся именно на английском)

In [19]:
replace_dict = {
    'шортс': 'shorts',
    "рутуб": "rutube",
    "рутубе": "rutube",
    "рутуба": "rutube",
    "рутубу": "rutube",
    "ютуб": "youtube",
    "ютубе": "youtube",
    "ютуба": "youtube",
    "ютубу": "youtube",
    "смарт": "smart",
    "тв": "tv",
    "самсунг": "samsung",
    "урл": "url",
}

### Загрузим предварительно обученные модели для будущей классификации вопросов

In [20]:
!cp -r drive/MyDrive/HacksAI/Russia24/models/version_20240928_121531_train_0.80_test_0.66_valid_0.61/ model_1/
!cp -r drive/MyDrive/HacksAI/Russia24/models/version_20240928_123836_train_0.45_test_0.41_valid_0.32/ model_2/

Вспомогательные функции для моделей классификации

In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import joblib

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('russian'))

loaded_models = {}
loaded_encoders = {}
loaded_vectorizers = {}

def preprocess(text):
    text = re.sub(r'\W+', ' ', text)

    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return ' '.join(words)

def load_model_and_files(model_name, model_path, encoder_path, vectorizer_path):
    global loaded_models, loaded_encoders, loaded_vectorizers

    if model_name not in loaded_models:
        print(f"Загрузка модели {model_name}...")
        loaded_models[model_name] = joblib.load(model_path)
        loaded_encoders[model_name] = joblib.load(encoder_path)
        loaded_vectorizers[model_name] = joblib.load(vectorizer_path)
    else:
        print(f"Модель {model_name} уже загружена.")

def classify_question(
    question: str,
    model_name: str,
):
    global loaded_models, loaded_encoders, loaded_vectorizers

    preprocessed_question = preprocess(question)

    # Преобразование вопроса в TF-IDF вектор
    vectorizer = loaded_vectorizers[model_name]
    question_tfidf = vectorizer.transform([preprocessed_question])

    # Предсказание с помощью модели
    model = loaded_models[model_name]
    predicted_label_numeric = model.predict(question_tfidf)

    # Преобразование числовой метки обратно в текстовую метку
    encoder = loaded_encoders[model_name]
    predicted_label_text = encoder.inverse_transform(predicted_label_numeric)

    return predicted_label_text[0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
model_path_1 = "model_1/classifier_1.pkl"
encoder_path_1 = "model_1/label_encoder_1.pkl"
vectorizer_path_1 = "model_1/vectorizer_1.pkl"

model_path_2 = "model_2/classifier_2.pkl"
encoder_path_2 = "model_2/label_encoder_2.pkl"
vectorizer_path_2 = "model_2/vectorizer_2.pkl"

load_model_and_files('model_1', model_path_1, encoder_path_1, vectorizer_path_1)
load_model_and_files('model_2', model_path_2, encoder_path_2, vectorizer_path_2)

Загрузка модели model_1...
Загрузка модели model_2...


### Интерфейс для получения ответа из RAG пайплайна

In [23]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def preprocess_text(text):
    """
    Функция предобработки текста: приведение к нижнему регистру, удаление лишних символов,
    лемматизация и замена слов по словарю, с сохранением знаков препинания.
    Если слово не найдено в replace_dict, оно остаётся в исходной форме.
    """
    # Токенизация: разделяем слова и знаки препинания
    tokens = re.findall(r'\w+|[^\w\s]', text.lower())
    processed_tokens = []

    for token in tokens:
        if re.match(r'\w+', token):  # Если это слово
            if replace_dict and token in replace_dict:
                # Заменяем слово, если оно есть в словаре
                processed_tokens.append(replace_dict[token])
            else:
                # Если нет в словаре замен, оставляем слово как есть
                processed_tokens.append(token)
        else:
            # Знак препинания остаётся без изменений
            processed_tokens.append(token)

    # Восстановление текста с правильным расположением знаков препинания
    processed_text = ''
    for i, token in enumerate(processed_tokens):
        if token in '.,!?;:':
            # Прикрепляем знак препинания к предыдущему слову без пробела
            processed_text = processed_text.rstrip() + token + ' '
        else:
            processed_text += token + ' '

    return processed_text.strip()

In [71]:
import re
import time
import pandas as pd

def get_answer_from_rag(
    question: str,
    basic_rag_pipeline,
    threshold: float = 0.35,
    df:pd.DataFrame = final_df,
    ):
    start_time = time.time()

    question = preprocess_text(question)
    print(f"Вопрос после очистки: {question}")

    response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "ranker": {"query": question}})
    print(response)

    top_documents = response['ranker']['documents']

    # Проверяем источник первого документа
    first_doc_source = top_documents[0].meta.get('source', '')

    if first_doc_source != "FAQ" or top_documents[0].score < threshold:
        modified_documents = []
        for doc in top_documents:
            if doc.meta.get('source') == "FAQ":
                target_q = doc.content
                # Поиск соответствующего ответа из БЗ
                matched_rows = df.loc[df['Вопрос'] == target_q]
                if not matched_rows.empty:
                    answer_bz = matched_rows.iloc[0]['Ответ из БЗ']
                    doc.content = answer_bz
                    modified_documents.append(doc)
                else:
                    # Если соответствующего вопроса нет в БЗ, оставляем оригинальное содержимое
                    modified_documents.append(doc)
            else:
                modified_documents.append(doc)

        print("Затраченное время:", time.time() - start_time)
        # Возвращаем список модифицированных документов
        return modified_documents
    else:
        # Существующая логика, когда источник первого документа "FAQ"
        document = top_documents[0]

        target_q = document.content
        score = document.score

        if score < threshold:
            answer_text = "Ответ не найден."

            class_1 = classify_question(question, "model_1")
            class_2 = classify_question(question, "model_2")

        elif document.meta.get('source') in ("Генеральное пользовательское соглашение", "Условия размещение контента"):
            answer_text = f"{document.content}\nИсточник: {document.meta.get('source', '')} {document.meta.get('idx', '')}"

            class_1 = classify_question(question, "model_1")
            class_2 = classify_question(question, "model_2")
        else:
            # Предполагается, что 'Вопрос' уникален в df
            matched_rows = df.loc[df['Вопрос'] == target_q]
            if not matched_rows.empty:
                target_answer = matched_rows.iloc[0]
                answer_text = target_answer['Ответ из БЗ']

                class_1 = target_answer.get('Классификатор 1 уровня', None)
                class_2 = target_answer.get('Классификатор 2 уровня', None)
            else:
                answer_text = "Ответ не найден."
                class_1 = classify_question(question, "model_1")
                class_2 = classify_question(question, "model_2")

        print("Затраченное время:", time.time() - start_time)

        return answer_text, class_1, class_2, score

In [77]:
import json
from typing import List
from pydantic import BaseModel

class Answer(BaseModel):
    answer: str
    class_1: str
    class_2: str
    score: float
    source: List

def process_rag_response(question, basic_rag_pipeline, pipe, threshold=0.5, df=final_df, GROUNDED_SYSTEM_PROMPT="Your task is to answer the user's questions using only the information from the provided documents. If the question has nothing to do with the context, just say that you have \"Нет ответа\"."):
    response = get_answer_from_rag(
        question=question,
        basic_rag_pipeline=basic_rag_pipeline,
        threshold=threshold,
        df=df
    )

    # Проверяем, является ли response кортежем
    if isinstance(response, tuple):
        answer_text, class_1, class_2, score = response
        return Answer(
            answer=answer_text,
            class_1=class_1,
            class_2=class_2,
            score=score,
            source=[{"type": "FAQ", "text": ""}]
        )

    # Проверяем, является ли response списком
    elif isinstance(response, list):
        source = []
        # Предполагается, что список содержит документы
        formatted_documents = []
        for idx, doc in enumerate(response):  # Берём первые три документа
            # Извлекаем заголовок из метаданных, если доступно
            title = doc.meta.get('title', f"Документ {idx + 1}")
            content = doc.content

            # Формируем словарь документа
            formatted_documents.append({
                "doc_id": idx,
                "title": title,
                "content": content
            })
            source.append( {
                "type": doc.meta['source'],
                "text": doc.meta['idx']
            })

        # Создаём sample_history для передачи в LLM модель
        sample_history = [
            {'role': 'system', 'content': GROUNDED_SYSTEM_PROMPT},
            {'role': 'documents', 'content': json.dumps(formatted_documents, ensure_ascii=False)},
            {'role': 'user', 'content': question}
        ]

        # Передаём sample_history в LLM модель и получаем ответ
        answer = pipe(sample_history)
        answer_text = answer[0]['generated_text'][-1]['content']

        return Answer(
            answer=answer_text,
            class_1="ОТСУТСТВУЕТ",
            class_2="Отсутствует",
            score=response[0].score,
            source=source)

### Асинхронная функция для будущего использования с API

In [80]:
import asyncio

async def get_answer(question: str, basic_rag_pipeline) -> Answer:
    """
    Асинхронная функция принимает вопрос, получает ответ и классы с помощью RAG pipeline
    и возвращает объект AnswerModel.

    Параметры:
    - question (str): Вопрос, который нужно обработать.
    - basic_rag_pipeline: RAG pipeline для обработки запроса.
    - replace_dict (dict, optional): Словарь для замены слов в вопросе.
    - df (pd.DataFrame, optional): DataFrame, содержащий столбцы 'Вопрос из БЗ', 'Ответ из БЗ',
                                    'Классификатор 1 уровня' и 'Классификатор 2 уровня'.

    Возвращает:
    - AnswerModel: Объект с полями answer, class_1 и class_2.
    """
    loop = asyncio.get_event_loop()
    curr_answer = await loop.run_in_executor(
        None,
        process_rag_response,
        question,
        basic_rag_pipeline,
        pipe
    )

    return curr_answer

In [82]:
question = "Что администрация может делать?"

my_answer = await get_answer(
    question,
    basic_rag_pipeline
)

my_answer

Вопрос после очистки: что администрация может делать?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'ranker': {'documents': [Document(id=47d032896e2da36619f107989ebc55460164356e2e72fa2c532b5ac7524bbb4a, content: 'При обращении Пользователя к Администрации, Администрация вправе запросить у Пользователя документы,...', meta: {'source': 'Генеральное пользовательское соглашение', 'idx': '1.7.'}, score: 0.6717730164527893), Document(id=c84cbe11323545feb00fd4363429c97de28c626ae4f0a4c5a6d55eadf81ac441, content: 'сколько человек можно сделать администраторами?', meta: {'source': 'FAQ', 'idx': 525}, score: 0.547361433506012), Document(id=0212788db2e1530325d0f6e090eed2555104be7a5720408a18adae4cea9d72f5, content: 'Какие возможности взаимодействия со Студией доступны пользователю с правами Редактора?', meta: {'source': 'FAQ', 'idx': 93}, score: 0.15697705745697021)]}}
Затраченное время: 0.3476588726043701


Answer(answer='Администрация имеет право запросить у Пользователя документы, подтверждающие информацию, указанную в обращении, и право Пользователя на направление такого обращения. Если Пользователь не предоставит запрошенные документы или информацию, Администрация может отказаться от осуществления действий, связанных с обращением Пользователя.\n\nКроме того, в контексте управления каналом, Администрация может устанавливать лимиты на количество пользователей с разными ролями, например, не более 50 человек для одного канала. Также Администрация может определять, какие действия могут выполнять пользователи с разными правами, например, пользователь с правами редактора может выполнять различные действия в Студии, но исключая удаление канала и управление настройками доступа к каналу.', class_1='ОТСУТСТВУЕТ', class_2='Отсутствует', score=0.6717730164527893, source=[{'type': 'Генеральное пользовательское соглашение', 'text': '1.7.'}, {'type': 'FAQ', 'text': 525}, {'type': 'FAQ', 'text': 93}])

### Тест модели

In [ ]:
import pandas as pd

def process_all_questions(input_file, question_col, basic_rag_pipeline, output_file):
    """
    Функция обрабатывает все вопросы из указанного файла, вызывает get_answer_from_rag для каждого,
    и сохраняет результаты в новый CSV файл.

    Параметры:
    - input_file: Путь к CSV или XLSX файлу с вопросами.
    - question_col: Название колонки, содержащей вопросы.
    - basic_rag_pipeline: RAG pipeline для обработки запроса.
    - output_file: Путь для сохранения результирующего CSV файла с вопросами и ответами.

    Возвращает:
    - DataFrame с исходными вопросами и найденными ответами.
    """

    file_extension = input_file.split('.')[-1].lower()

    # Чтение данных в зависимости от расширения
    if file_extension == 'csv':
        df_questions = pd.read_csv(input_file)
    elif file_extension == 'xlsx':
        df_questions = pd.read_excel(input_file)
    else:
        raise ValueError("Файл должен быть формата CSV или XLSX.")

    results = []

    for question in df_questions[question_col]:
        answer, class_1, class_2 = process_rag_response(question, basic_rag_pipeline, pipe)
        results.append({'Вопрос': question, 'Ответ': answer})

    df_results = pd.DataFrame(results)
    df_results.to_csv(output_file, index=False, encoding='utf-8')

    print(f"Результаты сохранены в файл: {output_file}")
    return df_results

In [ ]:
# main_path = "drive/MyDrive/HacksAI/Russia24/"

# src_file_path = main_path + "ground_truth_data/pipeline3_g_t_data.csv"
# out_file_path = main_path + "generated_data/pipeline3_answers.csv"

# process_all_questions(src_file_path, 'Вопрос', basic_rag_pipeline, out_file_path)